 STEP 1 – تعريف المشكلة و الهدف  <br>

**المشكلة** : اختيار الممثلين المناسبين لفلم جديد
<br>
**الهدف** : اقتراح 10 ممثلين مناسبين بالنظر الى الميزانية و نوع الفلم 

تحميل البيانات

In [ ]:

import pandas as pd
import numpy as np
import json
import random
from collections import defaultdict
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")
import gdown
import pandas as pd
import numpy as np
import faiss
import ast
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import ast


random.seed(42)

In [ ]:

file_id = "1fQ6NfLqpjuji_aEUjv_8lQZ1QffAHztt"  # بيانات الافلام
output_file = "downloaded_file.csv"  # 

gdown.download(id=file_id, output=output_file, quiet=False)

movies_df = pd.read_csv(output_file) 


Downloading...
From: https://drive.google.com/uc?id=1fQ6NfLqpjuji_aEUjv_8lQZ1QffAHztt
To: /content/downloaded_file.csv
100%|██████████| 46.7M/46.7M [00:00<00:00, 82.9MB/s]


'downloaded_file.csv'

In [ ]:




# -------------------------------
# استخراج اسماء الممثلين
# -------------------------------
def extract_actor_list(credits_str):
    
    return [actor.strip() for actor in credits_str.split('-') if actor.strip()]


movies_df['actor_list'] = movies_df['credits'].apply(extract_actor_list)

print(movies_df['actor_list'].iloc[0])


# استخراج الممثلين الفريدين
all_actors = list(set([a for actors in movies_df['actor_list'] for a in actors]))

# -------------------------------
# ✅ Step 2: Generate Movie–Actor Pairs
# توليد ازواج فلم-ممثل
# -------------------------------
positive_rows = []
negative_rows = []

def generate_negative_samples(row, n=5):
    actual = set(row['actor_list'])
    negatives = list(set(random.sample(all_actors, n + len(actual))) - actual)
    return negatives[:n]

for idx, row in movies_df.iterrows():
    for actor in row['actor_list']:
        positive_rows.append({'movie_id': row['id'], 'actor': actor, 'label': 1, **row.to_dict()})
    for neg_actor in generate_negative_samples(row):
        negative_rows.append({'movie_id': row['id'], 'actor': neg_actor, 'label': 0, **row.to_dict()})

pairs_df = pd.DataFrame(positive_rows + negative_rows)




['Jason Statham', 'Wu Jing', 'Shuya Sophia Cai', 'Sergio Peris', 'Mencheta', 'Skyler Samuels', 'Cliff Curtis', 'Page Kennedy', 'Sienna Guillory', 'Melissanthi Mahut', 'Kiran Sonia Sawar', 'Felix Mayr', 'Whoopie van Raam', 'Guo Tao', 'Robin Hill', 'Dai Lele', 'Sui Fong Ivy Tsui', 'Stewart Alexander', 'Li Xin', 'Billy Clements', 'Ron Smoorenburg', 'Rui Shang', 'Sara Dee', 'Jonny James', 'Bai Narisu', 'Kenneth Won', 'Able Wanamakok']


# Content-based filtering

التصفية المعتمدة على المحتوى  هي طريقة توصية تعتمد على خصائص العناصر (مثل نوع الفيلم، الميزانية، التصنيف).
مشكلتنا تُصنف كتصفية معتمدة على المحتوى لأننا نوصي بالممثلين بناءً على خصائص الفيلم الجديد.
لا نعتمد على تفضيلات المستخدمين، بل على تشابه الفيلم مع بيانات سابقة.
نقوم بتمثيل كل فيلم كمجموعة من الميزات ثم نبحث عن الممثلين الأكثر توافقًا مع هذه الميزات.

In [ ]:
movies_df.head()

,id,title,genres_x,original_language,overview,popularity,production_companies,release_date,budget,revenue,...,movie_age,rating_category,age_group,normalized_primary_title,has_superstar_actor,movie_oscar,normalized_credits_list,movie_credits_oscar,company_oscars,actor_list
0,615656,Meg 2: The Trench,Action-Science Fiction-Horror,en,An exploratory dive into the deepest depths of...,8763.998,Apelles Entertainment-Warner Bros. Pictures-di...,2023-08-02,129000000.0,3.520565e+08,...,2.0,Average,0–10 yrs,meg2thetrench,False,0,"['jasonstatham', 'wujing', 'shuyasophiacai', '...",0,0,"[Jason Statham, Wu Jing, Shuya Sophia Cai, Ser..."
1,758323,The Pope's Exorcist,Horror-Mystery-Thriller,en,Father Gabriele Amorth Chief Exorcist of the V...,5953.227,Screen Gems-2.0 Entertainment-Jesus & Mary-Wor...,2023-04-05,18000000.0,6.567582e+07,...,2.0,Average,0–10 yrs,thepopesexorcist,False,0,"['russellcrowe', 'danielzovatto', 'alexessoe',...",2,0,"[Russell Crowe, Daniel Zovatto, Alex Essoe, Fr..."
2,533535,Deadpool & Wolverine,Action-Comedy-Science Fiction,en,A listless Wade Wilson toils away in civilian ...,5410.496,Marvel Studios-Maximum Effort-21 Laps Entertai...,2024-07-24,200000000.0,1.326387e+09,...,1.0,Good,0–10 yrs,deadpoolwolverine,True,0,"['ryanreynolds', 'hughjackman', 'emmacorrin', ...",2,0,"[Ryan Reynolds, Hugh Jackman, Emma Corrin, Mat..."
3,667538,Transformers: Rise of the Beasts,Action-Adventure-Science Fiction,en,When a new threat capable of destroying the en...,5409.104,Skydance-Paramount-di Bonaventura Pictures-Bay...,2023-06-06,200000000.0,4.070455e+08,...,2.0,Average,0–10 yrs,transformersriseofthebeasts,False,0,"['anthonyramos', 'dominiquefishback', 'lunalau...",2,0,"[Anthony Ramos, Dominique Fishback, Luna Laure..."
4,693134,Dune: Part Two,Science Fiction-Adventure,en,Follow the mythic journey of Paul Atreides as ...,4742.163,Legendary Pictures,2024-02-27,190000000.0,6.838137e+08,...,1.0,Excellent,0–10 yrs,duneparttwo,False,2,"['timothéechalamet', 'zendaya', 'rebeccafergus...",2,4,"[Timothée Chalamet, Zendaya, Rebecca Ferguson,..."


In [ ]:
# كل ممثل له الصف الخاص به
exploded_df = movies_df.explode('actor_list')

#  توليد خصائص الممثلين
actor_profiles = exploded_df.groupby('actor_list').agg({
    'genres_x': lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0],
    'budget': 'mean',
    'revenue': 'mean',
    'vote_average': 'mean',
    'popularity': 'mean'
}).reset_index()

actor_profiles.rename(columns={
    'actors': 'actor_name',
    'genres_x': 'top_genre',
    'budget': 'avg_budget',
    'revenue': 'avg_revenue',
    'vote_average': 'avg_rating',
    'popularity': 'avg_popularity'
}, inplace=True)



# FAISS
فايس  هو مكتبة متقدمة للبحث عن التشابه في الفضاءات ذات الأبعاد العالية بسرعة كبيرة.
نستخدمه لأن لدينا تمثيلات عددية كبيرة للممثلين والأفلام (فيكتورات متعددة الأبعاد).
فايس يسمح لنا بإيجاد أقرب الممثلين لفيلم معين بناءً على التشابه في هذه التمثيلات.
هذه الطريقة فعالة جداً مقارنة بالحساب اليدوي لتشابه الكوزاين عبر جميع البيانات.
لذلك، استخدمنا فايس لتحسين سرعة ودقة نظام التوصية لدينا.


In [10]:
# Install FAISS (if not already in Colab)
!pip install faiss-cpu -q



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 55.3 MB/s eta 0:00:00


التجربة 1

يهدف هذا النموذج إلى التوصية بالممثلين بناءً على خصائص الفيلم باستخدام تقنية  للبحث عن التشابه. يقوم النموذج بإنشاء ملفات تعريف للممثلين تتضمن توزيع الأنواع وإحصائيات الميزانية والإيرادات. يتم تدريب الفهرس على بيانات التدريب، ويتم تقييم الأداء باستخدام مقاييس الدقة والاستدعاء و F1-score. أظهرت النتائج الأولية مستويات متفاوتة من الدقة والاستدعاء.

In [ ]:


# تقسيم الى بيانات تدريب و اختبار عشوائية
movies_df = movies_df.dropna(subset=['actor_list', 'genres_x', 'budget', 'vote_average', 'popularity'])

train_df, test_df = train_test_split(movies_df, test_size=0.2, random_state=42)

train_ids = set(train_df['id'])
train_exploded = exploded_df[exploded_df['id'].isin(train_ids)]

# خصائص الممثلين
genre_counts = pd.crosstab(train_exploded['actor_list'], train_exploded['genres_x'])
genre_dist = genre_counts.div(genre_counts.sum(axis=1), axis=0).fillna(0)

train_actor_stats = train_exploded.groupby('actor_list').agg({
    'budget': 'mean',
    'revenue': 'mean',
    'vote_average': 'mean',
    'popularity': 'mean'
}).rename(columns={
    'budget': 'avg_budget',
    'revenue': 'avg_revenue',
    'vote_average': 'avg_rating',
    'popularity': 'avg_popularity'
})

actor_profiles = train_actor_stats.join(genre_dist).fillna(0)

scaler = StandardScaler()
numeric_cols = ['avg_budget', 'avg_rating', 'avg_popularity']
actor_profiles[numeric_cols] = scaler.fit_transform(actor_profiles[numeric_cols])

genre_columns = genre_dist.columns.tolist()
actor_vectors = np.hstack([
    actor_profiles[genre_columns].values,
    actor_profiles[numeric_cols].values
])

actor_names = actor_profiles.index.to_list()

# بناء FAISS INDEX
actor_matrix = np.ascontiguousarray(actor_vectors.astype('float32'))
faiss.normalize_L2(actor_matrix)
index = faiss.IndexFlatIP(actor_matrix.shape[1])
index.add(actor_matrix)
print("✅ FAISS index built:", actor_matrix.flags['C_CONTIGUOUS'])

# تابع الاقتراحات
def recommend_actors_faiss(new_movie, actor_names, scaler, genre_columns, top_k=10):
    genre_vector = np.zeros(len(genre_columns), dtype='float32')
    genres = [g.strip() for g in new_movie['genres'].split('-')]
    for g in genres:
        if g in genre_columns:
            genre_vector[genre_columns.index(g)] = 1 / len(genres)

    numeric_input = scaler.transform([[new_movie['budget'], new_movie['vote_average'], new_movie['popularity']]])
    movie_vector = np.hstack([genre_vector, numeric_input.flatten().astype('float32')])
    movie_vector = np.ascontiguousarray(movie_vector.reshape(1, -1))
    faiss.normalize_L2(movie_vector)

    sim_scores, indices = index.search(movie_vector, top_k)
    return [actor_names[i] for i in indices.flatten()]

# تنظيف الاسماء
def clean_name(name):
    return str(name).strip().lower().replace('"', '').replace("'", '')

actor_names_clean = set(map(clean_name, actor_names))

# التقييم
def precision_recall_f1(recommended, actual, k):
    recommended_k = set(map(clean_name, recommended[:k]))
    actual_k = set(map(clean_name, actual))
    intersection = recommended_k & actual_k
    precision = len(intersection) / k
    recall = len(intersection) / len(actual_k) if actual_k else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1


precisions, recalls, f1s = [], [], []
skipped = 0

print("🔍 Evaluating on test set...")

for _, movie in test_df.iterrows():
    raw_actors = movie['actor_list']

    # Parse list safely
    if isinstance(raw_actors, str):
        try:
            actual_actors = ast.literal_eval(raw_actors)
        except:
            actual_actors = []
    else:
        actual_actors = raw_actors

    if not isinstance(actual_actors, list):
        continue

    actual_cleaned = [clean_name(a) for a in actual_actors]

    # Skip if no actors known
    if not any(a in actor_names_clean for a in actual_cleaned):
        skipped += 1
        continue

    movie_features = {
        'genres': movie['genres_x'],
        'budget': movie['budget'],
        'vote_average': movie['vote_average'],
        'popularity': movie['popularity']
    }

    recommended_actors = recommend_actors_faiss(
        movie_features, actor_names, scaler, genre_columns, top_k=10
    )

    print(f"🎬 {movie['title']}")
    print("Actual cleaned:", actual_cleaned)
    print("Recommended cleaned:", list(map(clean_name, recommended_actors)))
    print("Intersection:", set(actual_cleaned) & set(map(clean_name, recommended_actors)))

    p, r, f = precision_recall_f1(recommended_actors, actual_cleaned, k=10)
    precisions.append(p)
    recalls.append(r)
    f1s.append(f)

    print(f"🎯 P={p:.2f} R={r:.2f} F1={f:.2f}\n")

# === 8. Final Results ===
print("✅ Evaluation complete.")
print(f"📊 Evaluated on {len(precisions)} movies (Skipped: {skipped})")
print(f"🎯 Average Precision@10: {np.mean(precisions):.4f}")
print(f"🎯 Average Recall@10:    {np.mean(recalls):.4f}")
print(f"🎯 Average F1@10:        {np.mean(f1s):.4f}")


يُقدم هذا النموذج تحسينات على النموذج السابق، حيث يضيف مرونة في التوصية بالممثلين باستخدام وظيفتي FAISS منفصلتين لبيانات الفيلم الكاملة والبيانات الدنيا. كما يقوم بتصفية الممثلين الذين ظهروا في أكثر من 5 أفلام لتركيز التوصيات على الممثلين الأكثر نشاطاً، ويستخدم تشابه جيب التمام (Cosine Similarity) لتقييم مدى تطابق الممثلين الموصى بهم مع الممثلين الفعليين للفيلم.

In [ ]:

# actor_profiles.index = actor_profiles.index.map(lambda x: x.strip().lower())
actor_profiles.index = actor_profiles.index.map(
    lambda x: x.strip().lower() if isinstance(x, str) else x
)


def recommend_actors_faiss_full(movie, actor_names, scaler, genre_columns, top_k=10):
    """Used for movies with full metadata."""
    genre_vector = np.zeros(len(genre_columns), dtype='float32')
    genres = [g.strip() for g in movie['genres'].split('-')]
    for g in genres:
        if g in genre_columns:
            genre_vector[genre_columns.index(g)] = 1 / len(genres)

    numeric_input = scaler.transform([[movie['budget'], movie['vote_average'], movie['popularity']]])
    movie_vector = np.hstack([genre_vector, numeric_input.flatten().astype('float32')])
    movie_vector = np.ascontiguousarray(movie_vector.reshape(1, -1))
    faiss.normalize_L2(movie_vector)

    # sim_scores, indices = index.search(movie_vector, top_k)
    sim_scores, indices = index_full.search(movie_vector, top_k)
    return [actor_names[i] for i in indices.flatten()]

def recommend_actors_faiss_minimal_simple(new_movie, actor_names, genre_columns, scaler, top_k=10):
    genre_vector = np.zeros(len(genre_columns), dtype='float32')
    genres = [g.strip() for g in new_movie['genres'].split('-')]
    for g in genres:
        if g in genre_columns:
            genre_vector[genre_columns.index(g)] = 1 / len(genres)

    # تقييس الميزانية فقط
    budget_scaled = scaler.transform([[new_movie['budget'], 0, 0]])[0][0]  
    numeric_vector = np.array([budget_scaled], dtype='float32')

    movie_vector = np.hstack([genre_vector, numeric_vector])
    movie_vector = np.ascontiguousarray(movie_vector.reshape(1, -1))
    faiss.normalize_L2(movie_vector)

    # sim_scores, indices = index.search(movie_vector, top_k)
    sim_scores, indices = index_minimal.search(movie_vector, top_k)

    return [actor_names[i] for i in indices.flatten()]

def average_cosine_similarity(recommended, actual, actor_profiles):
    rec_clean = [a.strip().lower() for a in recommended if a.strip().lower() in actor_profiles.index]
    act_clean = [a.strip().lower() for a in actual if a.strip().lower() in actor_profiles.index]

    if not rec_clean or not act_clean:
        return None

    rec_vecs = actor_profiles.loc[rec_clean].values
    act_vecs = actor_profiles.loc[act_clean].values
    sim_matrix = cosine_similarity(rec_vecs, act_vecs)
    return np.mean(sim_matrix)

# حلقة التقييم , تختار بشكل تلقائي بين ال الكامل او المصغر
# اختيار الممثلين الذي لديهم اكثر من 5 افلام على الاقل
actor_counts = train_exploded['actor_list'].value_counts()
valid_actors = set(actor_counts[actor_counts > 5].index.str.strip().str.lower())

# Filter actor_profiles and actor_names
actor_profiles = actor_profiles.loc[actor_profiles.index.isin(valid_actors)]
actor_names = actor_profiles.index.tolist()
# actor_vectors = actor_profiles[genre_columns + ['avg_budget', 'avg_rating', 'avg_popularity']].values.astype('float32')
# المودل الكامل
actor_vectors_full = actor_profiles[genre_columns + ['avg_budget', 'avg_rating', 'avg_popularity']].values.astype('float32')
actor_vectors_full = np.ascontiguousarray(actor_vectors_full)
faiss.normalize_L2(actor_vectors_full)
index_full = faiss.IndexFlatIP(actor_vectors_full.shape[1])
index_full.add(actor_vectors_full)

# المودل المصغر
actor_vectors_minimal = actor_profiles[genre_columns + ['avg_budget']].values.astype('float32')
actor_vectors_minimal = np.ascontiguousarray(actor_vectors_minimal)
faiss.normalize_L2(actor_vectors_minimal)
index_minimal = faiss.IndexFlatIP(actor_vectors_minimal.shape[1])
index_minimal.add(actor_vectors_minimal)


actor_names = actor_profiles.index.to_list()


print(f"✅ Filtered to {len(actor_names)} actors who appeared in more than 5 movies.")
similarity_scores = []
skipped = 0
valid_movies = []  
for _, movie in test_df.iterrows():
    actual_actors = movie['actor_list']
    if isinstance(actual_actors, str):
        try:
            actual_actors = ast.literal_eval(actual_actors)
        except:
            continue
    if not isinstance(actual_actors, list):
        continue
    clean_actual = [a.strip().lower() for a in actual_actors]
    if any(a in actor_profiles.index for a in clean_actual):
        valid_movies.append((movie, clean_actual))

print(f"🔍 Preparing to evaluate {len(valid_movies)} movies using cosine similarity...")

for movie, actual_cleaned in valid_movies:
    movie_features = {
        'genres': movie['genres_x'],
        'budget': movie['budget'],
        'vote_average': movie.get('vote_average'),
        'popularity': movie.get('popularity')
    }

    # اختيار المودل المناسب
    if pd.notna(movie_features['vote_average']) and pd.notna(movie_features['popularity']):
        recommended_actors = recommend_actors_faiss_full(
            movie_features, actor_names, scaler, genre_columns, top_k=10
        )
    else:
        recommended_actors = recommend_actors_faiss_minimal_simple(
      movie_features, actor_names, genre_columns, scaler, top_k=10
  )

    sim_score = average_cosine_similarity(recommended_actors, actual_cleaned, actor_profiles)

    if sim_score is not None:
        similarity_scores.append(sim_score)
        print(f"🎬 {movie['title']} | 🧠 Avg Cosine Similarity: {sim_score:.4f} | Recommended: {recommended_actors}")
    else:
        skipped += 1
        print(f"⚠️ Skipping {movie['title']} due to missing actor vectors")

print(f"\n✅ Evaluation complete.")
print(f"🎯 Avg Cosine Similarity: {np.mean(similarity_scores):.4f}")
print(f"🧠 Tested on {len(similarity_scores)} movies (Skipped: {skipped})")


# تقييم التشابه المبني على التضمين
Embedding-Based Similarity Evaluation


## استخدام استراتيجيات توصية منفصلة للأفلام الصادرة مقابل الأفلام الجديدة:
* للأفلام الصادرة: استخدم مجموعة الميزات الكاملة (الأنواع، الميزانية، متوسط التصويت، الشعبية).

* للأفلام الجديدة: استخدم فقط الأنواع والميزانية أو الأنواع فقط.

يقوم هذا النموذج بتوسيع النهج السابق عن طريق تحسين استراتيجية التوصية للممثلين. يحتفظ بالاستراتيجيات المنفصلة للأفلام ذات البيانات الكاملة والبيانات الدنيا، ويُعيد بناء فهارس FAISS بناءً على ملفات تعريف الممثلين المفلترة. يظل التركيز على تقييم تشابه جيب التمام، مما يوفر مقياساً شاملاً لمدى ملاءمة الممثلين الموصى بهم مقارنة بالممثلين الفعليين.

In [ ]:
actor_profiles.index = actor_profiles.index.map(
    lambda x: x.strip().lower() if isinstance(x, str) else x
)

def recommend_actors_faiss_full(movie, actor_names, scaler, genre_columns, top_k=10):
    """Used for movies with full metadata."""
    genre_vector = np.zeros(len(genre_columns), dtype='float32')
    genres = [g.strip() for g in movie['genres'].split('-')]
    for g in genres:
        if g in genre_columns:
            genre_vector[genre_columns.index(g)] = 1 / len(genres)

    numeric_input = scaler.transform([[movie['budget'], movie['vote_average'], movie['popularity']]])
    movie_vector = np.hstack([genre_vector, numeric_input.flatten().astype('float32')])
    movie_vector = np.ascontiguousarray(movie_vector.reshape(1, -1))
    faiss.normalize_L2(movie_vector)

    # sim_scores, indices = index.search(movie_vector, top_k)
    sim_scores, indices = index_full.search(movie_vector, top_k)
    return [actor_names[i] for i in indices.flatten()]

def recommend_actors_faiss_minimal_simple(new_movie, actor_names, genre_columns, scaler, top_k=10):
    genre_vector = np.zeros(len(genre_columns), dtype='float32')
    genres = [g.strip() for g in new_movie['genres'].split('-')]
    for g in genres:
        if g in genre_columns:
            genre_vector[genre_columns.index(g)] = 1 / len(genres)

    budget_scaled = scaler.transform([[new_movie['budget'], 0, 0]])[0][0] 
    numeric_vector = np.array([budget_scaled], dtype='float32')

    movie_vector = np.hstack([genre_vector, numeric_vector])
    movie_vector = np.ascontiguousarray(movie_vector.reshape(1, -1))
    faiss.normalize_L2(movie_vector)

    # sim_scores, indices = index.search(movie_vector, top_k)
    sim_scores, indices = index_minimal.search(movie_vector, top_k)

    return [actor_names[i] for i in indices.flatten()]

# ========== Cosine Similarity Evaluator ==========
def average_cosine_similarity(recommended, actual, actor_profiles):
    rec_clean = [a.strip().lower() for a in recommended if a.strip().lower() in actor_profiles.index]
    act_clean = [a.strip().lower() for a in actual if a.strip().lower() in actor_profiles.index]

    if not rec_clean or not act_clean:
        return None

    rec_vecs = actor_profiles.loc[rec_clean].values
    act_vecs = actor_profiles.loc[act_clean].values
    sim_matrix = cosine_similarity(rec_vecs, act_vecs)
    return np.mean(sim_matrix)

actor_counts = train_exploded['actor_list'].value_counts()
valid_actors = set(actor_counts[actor_counts > 5].index.str.strip().str.lower())
actor_profiles = actor_profiles.loc[actor_profiles.index.isin(valid_actors)]
actor_names = actor_profiles.index.tolist()

actor_vectors_full = actor_profiles[genre_columns + ['avg_budget', 'avg_rating', 'avg_popularity']].values.astype('float32')
actor_vectors_full = np.ascontiguousarray(actor_vectors_full)
faiss.normalize_L2(actor_vectors_full)
index_full = faiss.IndexFlatIP(actor_vectors_full.shape[1])
index_full.add(actor_vectors_full)

actor_vectors_minimal = actor_profiles[genre_columns + ['avg_budget']].values.astype('float32')
actor_vectors_minimal = np.ascontiguousarray(actor_vectors_minimal)
faiss.normalize_L2(actor_vectors_minimal)
index_minimal = faiss.IndexFlatIP(actor_vectors_minimal.shape[1])
index_minimal.add(actor_vectors_minimal)


actor_names = actor_profiles.index.to_list()


print(f"✅ Filtered to {len(actor_names)} actors who appeared in more than 5 movies.")
similarity_scores = []
skipped = 0
valid_movies = [] 
for _, movie in test_df.iterrows():
    actual_actors = movie['actor_list']
    if isinstance(actual_actors, str):
        try:
            actual_actors = ast.literal_eval(actual_actors)
        except:
            continue
    if not isinstance(actual_actors, list):
        continue
    clean_actual = [a.strip().lower() for a in actual_actors]
    if any(a in actor_profiles.index for a in clean_actual):
        valid_movies.append((movie, clean_actual))

print(f"🔍 Preparing to evaluate {len(valid_movies)} movies using cosine similarity...")

for movie, actual_cleaned in valid_movies:
    movie_features = {
        'genres': movie['genres_x'],
        'budget': movie['budget'],
        'vote_average': movie.get('vote_average'),
        'popularity': movie.get('popularity')
    }

    if pd.notna(movie_features['vote_average']) and pd.notna(movie_features['popularity']):
        recommended_actors = recommend_actors_faiss_full(
            movie_features, actor_names, scaler, genre_columns, top_k=10
        )
    else:
        recommended_actors = recommend_actors_faiss_minimal_simple(
      movie_features, actor_names, genre_columns, scaler, top_k=10
  )

    sim_score = average_cosine_similarity(recommended_actors, actual_cleaned, actor_profiles)

    if sim_score is not None:
        similarity_scores.append(sim_score)
        print(f"🎬 {movie['title']} | 🧠 Avg Cosine Similarity: {sim_score:.4f} | Recommended: {recommended_actors}")
    else:
        skipped += 1
        print(f"⚠️ Skipping {movie['title']} due to missing actor vectors")

print(f"\n✅ Evaluation complete.")
print(f"🎯 Avg Cosine Similarity: {np.mean(similarity_scores):.4f}")
print(f"🧠 Tested on {len(similarity_scores)} movies (Skipped: {skipped})")


✅ Filtered to 20690 actors who appeared in more than 5 movies.
🔍 Preparing to evaluate 6570 movies using cosine similarity...
🎬 Weathered | 🧠 Avg Cosine Similarity: 0.2000 | Recommended: ['anne schaefer', 'warren cook', 'david powell', 'mabel julienne scott', 'thomas meighan', 'corinne griffith', 'april hannah', 'willard louis', 'gertrude claire', 'joelean']
🎬 Madcap Mabel | 🧠 Avg Cosine Similarity: 0.9000 | Recommended: ['odessa young', 'katherine squire', 'nicholas jones', 'jim moriarty', 'frank maxwell', 'owen campbell', 'stacy martin', 'amelia campbell', 'ruby dee', 'richard kohnke']
🎬 The Manor | 🧠 Avg Cosine Similarity: 0.5000 | Recommended: ['jill haworth', 'tristan risk', 'devanny pinn', 'megan purvis', 'abi casson thompson', 'josh hammond', 'gunnar hansen', 'jeffrey reddick', 'ricardo freitas', 'melissa heflin']
🎬 Green Briefs | 🧠 Avg Cosine Similarity: 0.3000 | Recommended: ['william conklin', 'jan duggan', 'john patrick', 'tyrone power sr.', 'kate toncray', 'irene rich', 'wa

KeyboardInterrupt: 

ملاحظة : تم ايقاف التشغيل بشكل يدوي حتى لايقوم بطباعة كل القيم , الكود يعمل

---
تختلف الطرق الثلاث في تعقيد ميزات الأفلام المستخدمة للتوصية وفي منهجية تقييم الأداء. تركز الطريقة الأولى على التقييم المباشر لمقاييس الدقة والاستدعاء و F1-score. بينما تقدم الطريقتان الثانية والثالثة مرونة أكبر في التعامل مع بيانات الأفلام المتاحة (كاملة أو جزئية) وتعتمدان على تشابه جيب التمام لتقييم جودة التوصيات، مع تحسين تصفية الممثلين في الطريقة الثانية والثالثة.

| الميزة / الطريقة | الطريقة الأولى | الطريقة الثانية | الطريقة الثالثة |
|---|---|---|---|
| **ميزات الفيلم للتوصية** | ميزات كاملة (الأنواع، الميزانية، متوسط التصويت، الشعبية) | ميزات كاملة أو دنيا (الأنواع والميزانية فقط) | ميزات كاملة أو دنيا (الأنواع والميزانية فقط) |
| **تقنية التوصية** | FAISS للبحث عن التشابه | FAISS (مع فهرسين منفصلين) | FAISS (مع فهرسين منفصلين) |
| **تصفية الممثلين** | لا يوجد تصفية واضحة للممثلين | تصفية الممثلين ذوي الظهور > 5 | تصفية الممثلين ذوي الظهور > 5 |
| **مقياس التقييم الرئيسي** | الدقة، الاستدعاء، F1-score | متوسط تشابه جيب التمام | متوسط تشابه جيب التمام |
| **المرونة (بيانات الفيلم)** | أقل مرونة | مرونة أكبر للتعامل مع البيانات الناقصة | مرونة أكبر للتعامل مع البيانات الناقصة |
---

In [ ]:
# من اجل الطباعة و حفظ النتائج
actor_profiles.index = actor_profiles.index.map(lambda x: x.strip().lower())


actor_profiles.index = actor_profiles.index.map(lambda x: x.strip().lower())

results_table = []
similarity_scores = []
skipped = 0

print(f"🔍 Preparing to evaluate {len(valid_movies)} movies using cosine similarity...")

for movie, actual_actors in valid_movies:
    movie_features = {
        'genres': movie['genres_x'],
        'budget': movie['budget'],
        'vote_average': movie.get('vote_average'),
        'popularity': movie.get('popularity')
    }

    try:
        if pd.notna(movie_features['vote_average']) and pd.notna(movie_features['popularity']):
            recommended_actors = recommend_actors_faiss_full(
                movie_features, actor_names, scaler, genre_columns, top_k=10)
        else:
            recommended_actors = recommend_actors_faiss_minimal(
                movie_features, actor_names, genre_columns, scaler, top_k=10)

        sim_score = average_cosine_similarity(recommended_actors, actual_actors, actor_profiles)

        if sim_score is not None:
            similarity_scores.append(sim_score)
            results_table.append({
                'title': movie['title'],
                'similarity': sim_score,
                'recommended_actors': recommended_actors
            })
        else:
            skipped += 1
    except Exception as e:
        skipped += 1

results_df = pd.DataFrame(results_table)
results_df.to_csv("cosine_similarity_evaluation.csv", index=False)

print(f"\n✅ Evaluation complete. Saved results to 'cosine_similarity_evaluation.csv'")
print(f"🔢 Movies evaluated: {len(similarity_scores)} | Skipped: {skipped}")
print(f"🧠 Avg Cosine Similarity: {np.mean(similarity_scores):.4f}")


🔍 Preparing to evaluate 6570 movies using cosine similarity...

✅ Evaluation complete. Saved results to 'cosine_similarity_evaluation.csv'
🔢 Movies evaluated: 6570 | Skipped: 0
🧠 Avg Cosine Similarity: 0.6119


## الخبرة البشرية 
 Human expert

طريقة اخرى للتقييم هي استخدام الخبرة البشرية لتقييم اذا كان الممثلين مناسبين او لا 

In [ ]:
# انواع الافلام
print("🎭 genre_columns (from training data):")
print(genre_columns)


🎭 genre_columns (from training data):
['Action', 'Action-Adventure', 'Action-Adventure-Animation', 'Action-Adventure-Animation-Comedy-Family', 'Action-Adventure-Animation-Family', 'Action-Adventure-Animation-Fantasy-Science Fiction', 'Action-Adventure-Animation-Science Fiction', 'Action-Adventure-Comedy', 'Action-Adventure-Comedy-Crime', 'Action-Adventure-Comedy-Crime-Thriller', 'Action-Adventure-Comedy-Drama', 'Action-Adventure-Comedy-Drama-Romance', 'Action-Adventure-Comedy-Family', 'Action-Adventure-Comedy-Fantasy', 'Action-Adventure-Comedy-Horror-Science Fiction-Thriller', 'Action-Adventure-Comedy-Romance', 'Action-Adventure-Comedy-Science Fiction', 'Action-Adventure-Comedy-Thriller', 'Action-Adventure-Comedy-Western', 'Action-Adventure-Crime', 'Action-Adventure-Crime-Drama', 'Action-Adventure-Crime-Drama-Romance', 'Action-Adventure-Crime-Drama-Thriller', 'Action-Adventure-Crime-Mystery-Thriller', 'Action-Adventure-Crime-Thriller', 'Action-Adventure-Drama', 'Action-Adventure-Drama-

In [ ]:
# افلام جديدة يفكر بطرحها
custom_movies = [
    {
        'title': 'a comedy movie 1',
        'genres': 'Action',
        'budget': 50_000_000  
    },
    {
        'title': 'another comedy movie',
        'genres': 'Comedy',
        'budget': 80_000_000
    },
    {
        'title': 'an action movie',
        'genres': 'Romance',
        'budget': 200_000_000
    },
    {
        'title': 'funny action',
        'genres': 'Action-Comedy',
        'budget': 100_000_000
    }
]
print("🌟 Testing recommendations on custom movies 🌟\n")

for movie in custom_movies:
    print(f"🎬 {movie['title'].upper()}")
    print(f"📂 Genres: {movie['genres']} | 💰 Budget: ${movie['budget']:,}")

    recommended_actors = recommend_actors_faiss_minimal_simple(
        movie,
        actor_names,
        genre_columns,
        scaler,  
        top_k=10
    )

    print("👤 Recommended Actors:")
    for i, actor in enumerate(recommended_actors, start=1):
        print(f"  {i}. {actor}")
    print("📝 Your opinion? (Are they suitable?)")
    print("-" * 50)
for movie in custom_movies:
    print(f"\n🎬 {movie['title']}")
    genres = [g.strip() for g in movie['genres'].split('-')]
    print(f"Parsed genres: {genres}")
    genre_vector = np.zeros(len(genre_columns), dtype='float32')
    for g in genres:
        if g in genre_columns:
            genre_vector[genre_columns.index(g)] = 1 / len(genres)
        else:
            print(f"⚠️ Genre not found in training data: {g}")
    print(f"Genre vector: {genre_vector}")

🌟 Testing recommendations on custom movies 🌟

🎬 A COMEDY MOVIE 1
📂 Genres: Action | 💰 Budget: $50,000,000
👤 Recommended Actors:
  1. chris adams
  2. lee charles
  3. claude van damme
  4. dennis keiffer
  5. jeff imada
  6. anna van hooft
  7. george cheung
  8. vladimir kulich
  9. ego mikitas
  10. jacqueline obradors
📝 Your opinion? (Are they suitable?)
--------------------------------------------------
🎬 ANOTHER COMEDY MOVIE
📂 Genres: Comedy | 💰 Budget: $80,000,000
👤 Recommended Actors:
  1. sacha baron cohen
  2. bobby tisdale
  3. jim carrey
  4. brian hayes currie
  5. sam richardson
  6. googy gress
  7. nick kroll
  8. adam sandler
  9. eric andré
  10. randall park
📝 Your opinion? (Are they suitable?)
--------------------------------------------------
🎬 AN ACTION MOVIE
📂 Genres: Romance | 💰 Budget: $200,000,000
👤 Recommended Actors:
  1. chris hemsworth
  2. terry notary
  3. stan lee
  4. zoe saldaña
  5. winston duke
  6. vin diesel
  7. gal gadot
  8. terri douglas
  9. e

1. فلم الاكشن 
* تم اقتراح ممثلين , بعضهم بعيد و بعضهم مناسب جدا مثل van damme
2. فلم كوميدي 
* هنا الاقتراحات كانت افضل بكثير , فهنالك <br>
sacha , jim carrey , adam sandler ,eric andre <br>
و هؤلاء ممثلين مناسبين لفلم كوميدي 

3. فلم رومانسي 
* هنا ايضا الاقتراحات كانت جيدة , فهنالك <br>
elizabeth olsen & zoe saldana 
4. فلم اكشن كوميدي 
* cobie smulders & matt damon & ken jeong  <br>
يمكننا ان نرى فلم ممتع مع هؤلاء 


In [ ]:
df_eval = pd.read_csv("cosine_similarity_evaluation.csv")

print(df_eval.head())


          title  similarity                                 recommended_actors
0     Weathered         0.2  ['anne schaefer', 'warren cook', 'david powell...
1  Madcap Mabel         0.9  ['odessa young', 'katherine squire', 'nicholas...
2     The Manor         0.5  ['jill haworth', 'tristan risk', 'devanny pinn...
3  Green Briefs         0.3  ['william conklin', 'jan duggan', 'john patric...
4    Here Today         1.0  ['mark hadfield', 'william duell', 'peter bark...
